<a id="table-of-contents"></a>

<div style="background:#2b6684;color:white; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:30px;color:white"><u>Chapter 4</u></div>
<div style="background:#2b6684;color:white; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:30px;color:white"><u>Evaluation</u>    
<p style="font-family:cursive;font-size:17px; color:white" >Notebook is a part of FREE ML course by Glexey Grigorev
<a style="font-family:cursive;font-size:17px; color: yellow" href="https://datatalks.club/courses/2021-winter-ml-zoomcamp.html" target="_blank"> <u>Link for the Course</u></a></p>
<li style = "line-height: 0.7"><a style="font-family:cursive;font-size:17px; color:#ecfe15" href = "#1">1. Getting ready for Evaluation</a></li>   
<li style = "line-height: 0.7"><a style="font-family:cursive;font-size:17px; color:#ecfe15" href = "#2">2. Accuracy and dummy model</a></li> 
<li style = "line-height: 0.7"><a style="font-family:cursive;font-size:17px; color:#ecfe15" href = "#3">3. Confusion Matrix </a></li>
<li style = "line-height: 0.7"><a style="font-family:cursive;font-size:17px; color:#ecfe15" href = "#4"> 4. Precision and Recall  </a></li> 
<li style = "line-height: 0.7"><a style="font-family:cursive;font-size:17px; color:#ecfe15" href = "#5"> 5. ROC Curves </a></li>  
    <ul>
<li style = "line-height: 0.7"><a style="font-family:cursive;font-size:17px; color:#ecfe15" href = "#5.1"> 5.1 Random model</a></li>
<li style = "line-height: 0.7"><a style="font-family:cursive;font-size:17px; color:#ecfe15" href = "#5.2"> 5.2 Ideal model</a></li> 
<li style = "line-height: 0.7"><a style="font-family:cursive;font-size:17px; color:#ecfe15" href = "#5.3"> 5.3 Putting everything together</a></li>             
</ul>
<li style = "line-height: 0.7"><a style="font-family:cursive;font-size:17px; color:#ecfe15" href = "#6"> 6. ROC AUC</a></li> 
<li style = "line-height: 0.7"><a style="font-family:cursive;font-size:17px; color:#ecfe15" href = "#7">7. Cross-Validation</a></li>     
    
</div>    

[back to top](#table-of-contents)
<a id="1"></a>

<div style="background:#2b6684;color:white; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:25px;color:yellow">1. Getting ready for Evaluation
    
</div>

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import auc

plt.rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')

In [ ]:
df = pd.read_csv('../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)


df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

del df_train['churn']
del df_val['churn']
del df_test['churn']

numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]


dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

model = LogisticRegression()
model.fit(X_train, y_train)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

y_pred = model.predict_proba(X_val)[:, 1]
churn_decision = (y_pred >= 0.5)
(y_val == churn_decision).mean()

[back to top](#table-of-contents)
<a id="2"></a>

<div style="background:#2b6684;color:white; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:25px;color:yellow">2. Accuracy and dummy model
    
</div>

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_val, y_pred >= 0.5)

In [ ]:
"""Let's check the accuracy with different threshold"""
thresholds = np.linspace(0, 1, 21)

scores = []

for t in thresholds:
    score = accuracy_score(y_val, y_pred >= t)
    print('%.2f %.3f' % (t, score))
    scores.append(score)

In [ ]:
"""Let's plot these results"""
plt.plot(thresholds, scores);

In [ ]:
"""Also, our data is not balanced"""
df['churn'].mean()

**Observation:**
- **We can see that when all the predictions are False our model gives accuracy of ``0.726``**
- **This is because our data is ``not balanced`` . One class is way more than in numbers than the other.**
 

[back to top](#table-of-contents)
<a id="3"></a>

<div style="background:#2b6684;color:white; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:25px;color:yellow">3. Confusion Matrix
    
</div>

In [ ]:
actual_positive = (y_val == 1)
actual_negative = (y_val == 0)

t = 0.5
predict_positive = (y_pred >= t)
predict_negative = (y_pred < t)

tp = (predict_positive & actual_positive).sum()
tn = (predict_negative & actual_negative).sum()

fp = (predict_positive & actual_negative).sum()
fn = (predict_negative & actual_positive).sum()

confusion_matrix = np.array([
    [tn, fp],
    [fn, tp]
])
"""Confusion Matrix in frequency Form"""
confusion_matrix

In [ ]:
"""Confusion Matrix in percentage Form"""
(confusion_matrix / confusion_matrix.sum()).round(2)

[back to top](#table-of-contents)
<a id="4"></a>

<div style="background:#2b6684;color:white; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:25px;color:yellow">4. Precision and Recall
    
</div>

In [ ]:
"""Precision"""
p = tp / (tp + fp)
print(p)
"""Recall"""
r = tp / (tp + fn)
print(r)

[back to top](#table-of-contents)
<a id="5"></a>

<div style="background:#2b6684;color:white; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:25px;color:yellow">5. ROC Curves
    
</div>




In [ ]:
"""TPR"""
tpr = tp / (tp + fn)
print(tpr)
"""FRP"""
fpr = fp / (fp + tn)
print(fpr)

In [ ]:
scores = []

thresholds = np.linspace(0, 1, 101)

for t in thresholds:
    actual_positive = (y_val == 1)
    actual_negative = (y_val == 0)
    
    predict_positive = (y_pred >= t)
    predict_negative = (y_pred < t)

    tp = (predict_positive & actual_positive).sum()
    tn = (predict_negative & actual_negative).sum()

    fp = (predict_positive & actual_negative).sum()
    fn = (predict_negative & actual_positive).sum()
    
    scores.append((t, tp, fp, fn, tn))
    


columns = ['threshold', 'tp', 'fp', 'fn', 'tn']
df_scores = pd.DataFrame(scores, columns=columns)

df_scores['tpr'] = df_scores.tp / (df_scores.tp + df_scores.fn)
df_scores['fpr'] = df_scores.fp / (df_scores.fp + df_scores.tn)




plt.plot(df_scores.threshold, df_scores['tpr'], label='TPR')
plt.plot(df_scores.threshold, df_scores['fpr'], label='FPR')
plt.legend();
    

In [ ]:
df_scores

[back to top](#table-of-contents)
<a id="5.1"></a>

<div style="background:#59a1c6;color:black; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:25px">5.1 Random model
    
</div>

In [ ]:
"""Random model"""
np.random.seed(1)
y_rand = np.random.uniform(0, 1, size=len(y_val))

((y_rand >= 0.5) == y_val).mean()

In [ ]:
def tpr_fpr_dataframe(y_val, y_pred):
    scores = []

    thresholds = np.linspace(0, 1, 101)

    for t in thresholds:
        actual_positive = (y_val == 1)
        actual_negative = (y_val == 0)

        predict_positive = (y_pred >= t)
        predict_negative = (y_pred < t)

        tp = (predict_positive & actual_positive).sum()
        tn = (predict_negative & actual_negative).sum()

        fp = (predict_positive & actual_negative).sum()
        fn = (predict_negative & actual_positive).sum()

        scores.append((t, tp, fp, fn, tn))

    columns = ['threshold', 'tp', 'fp', 'fn', 'tn']
    df_scores = pd.DataFrame(scores, columns=columns)

    df_scores['tpr'] = df_scores.tp / (df_scores.tp + df_scores.fn)
    df_scores['fpr'] = df_scores.fp / (df_scores.fp + df_scores.tn)
    
    return df_scores

In [ ]:
df_rand = tpr_fpr_dataframe(y_val, y_rand)

plt.plot(df_rand.threshold, df_rand['tpr'], label='TPR')
plt.plot(df_rand.threshold, df_rand['fpr'], label='FPR')
plt.legend();

[back to top](#table-of-contents)
<a id="5.2"></a>

<div style="background:#59a1c6;color:black; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:25px">5.2 Ideal model
    
</div>

In [ ]:
"""Ideal model"""
num_neg = (y_val == 0).sum()
num_pos = (y_val == 1).sum()
num_neg, num_pos

y_ideal = np.repeat([0, 1], [num_neg, num_pos])

y_ideal_pred = np.linspace(0, 1, len(y_val))

In [ ]:
1 - y_val.mean()

In [ ]:
accuracy_score(y_ideal, y_ideal_pred >= 0.726)

In [ ]:
df_ideal = tpr_fpr_dataframe(y_ideal, y_ideal_pred)
df_ideal[::10]

In [ ]:
plt.plot(df_ideal.threshold, df_ideal['tpr'], label='TPR')
plt.plot(df_ideal.threshold, df_ideal['fpr'], label='FPR')
plt.legend()

[back to top](#table-of-contents)
<a id="5.3"></a>

<div style="background:#59a1c6;color:black; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:25px">5.3 Putting everything together
    
</div>

In [ ]:
"""Putting everything together"""
plt.plot(df_scores.threshold, df_scores['tpr'], label='TPR', color='black')
plt.plot(df_scores.threshold, df_scores['fpr'], label='FPR', color='blue')

plt.plot(df_ideal.threshold, df_ideal['tpr'], label='TPR ideal')
plt.plot(df_ideal.threshold, df_ideal['fpr'], label='FPR ideal')

# plt.plot(df_rand.threshold, df_rand['tpr'], label='TPR random', color='grey')
# plt.plot(df_rand.threshold, df_rand['fpr'], label='FPR random', color='grey')

plt.legend();

In [ ]:
plt.plot(df_scores.fpr, df_scores.tpr, label='Model')
plt.plot([0, 1], [0, 1], label='Random', linestyle='--')

plt.xlabel('FPR')
plt.ylabel('TPR')

plt.legend();

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_val, y_pred)

plt.plot(fpr, tpr, label='Model')
plt.plot([0, 1], [0, 1], label='Random', linestyle='--')

plt.xlabel('FPR')
plt.ylabel('TPR')

plt.legend();

[back to top](#table-of-contents)
<a id="6"></a>

<div style="background:#2b6684;color:white; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:25px;color:yellow">6. AUC
    
</div>

In [ ]:
"""Area under Curve basically calculates the probability that a randomly selected positive class has higher score than randomly selected negative class"""
print(auc(fpr, tpr))  # Score from implementation
print(auc(df_scores.fpr, df_scores.tpr)) #score from sklearn model

In [ ]:
auc(df_ideal.fpr, df_ideal.tpr)  # score from ideal model

In [ ]:
"""Calculating fpr and tpr using roc_curve from sklearn"""
fpr, tpr, thresholds = roc_curve(y_val, y_pred)
auc(fpr, tpr)                # Then calculating auc from fpr and tpr generated from previous step

In [ ]:
"""We can use sklearn roc_auc to calculate area under curve directly"""
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, y_pred)

In [ ]:
"""As discussed earlier: Area under Curve basically calculates the probability that a randomly selected positive class has higher score than 
randomly selected negative class

Let's implement that and see."""

neg = y_pred[y_val == 0]   # creating a variable negative to contain all -ve probabilities
pos = y_pred[y_val == 1]   # creating a variable positive to contain all +ve probabilities

In [ ]:
"""
Randomly selecting postive and negetive class and checking. 
If we get postive probability greater than negative probability we increment the score by 1
Finally we take the fraction by dividing the success by total number of times we repeated experiment. 
We did the experiment 100000 times.
"""
import random
n = 100000
success = 0 

for i in range(n):
    pos_ind = random.randint(0, len(pos) - 1)
    neg_ind = random.randint(0, len(neg) - 1)

    if pos[pos_ind] > neg[neg_ind]:
        success = success + 1

success / n

In [ ]:
"""Implementing it second time"""
n = 50000

np.random.seed(1)
pos_ind = np.random.randint(0, len(pos), size=n)
neg_ind = np.random.randint(0, len(neg), size=n)

(pos[pos_ind] > neg[neg_ind]).mean()


[back to top](#table-of-contents)
<a id="7"></a>

<div style="background:#2b6684;color:white; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:25px;color:yellow">7. Cross-Validation
    
</div>

In [ ]:
"""
Evaluating the same model on different subsets of data
Getting the average prediction and the spread within predictions
"""

def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)
    
    return dv, model

def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [ ]:
!pip install tqdm

In [ ]:
dv, model = train(df_train, y_train, C=0.001)
y_pred = predict(df_val, dv, model)

In [ ]:
from sklearn.model_selection import KFold
from tqdm.auto import tqdm

In [ ]:
n_splits = 5

for C in tqdm([0.001, 0.01, 0.1, 0.5, 1, 5, 10]):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

    scores = []

    for train_idx, val_idx in kfold.split(df_full_train):
        df_train = df_full_train.iloc[train_idx]
        df_val = df_full_train.iloc[val_idx]

        y_train = df_train.churn.values
        y_val = df_val.churn.values

        dv, model = train(df_train, y_train, C=C)
        y_pred = predict(df_val, dv, model)

        auc = roc_auc_score(y_val, y_pred)
        scores.append(auc)

    print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

In [ ]:
scores

In [ ]:
"""Now that we have the best parameter. Let's train the model using that"""
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)

auc = roc_auc_score(y_test, y_pred)
auc